In [1]:
try:
    import google.colab
    !rm -rf /content/SportWarzSim
    !git clone https://github.com/daltontf/SportWarzSim.git 
    %cd SportWarzSim
except ImportError:
    pass

In [2]:
from sportwarz import load_counties_geojson

counties_geojson = load_counties_geojson()

In [3]:
from sportwarz import load_counties_data

co_data_frame = load_counties_data(counties_geojson)

In [4]:
from sportwarz import load_leagues

leagues = {
    'NFL': {  },
    'NBA': {  },
    'MLB': {  },
    'NHL': {  }, 
    'MLS': {  }
}    

load_leagues(leagues)

In [5]:
from sportwarz import calculate_distances

calculate_distances(leagues, co_data_frame) 

In [6]:
from sportwarz import compute_shares
        
compute_shares(leagues, co_data_frame)  

In [7]:
from sportwarz import compute_output_dataframes

compute_output_dataframes(leagues, co_data_frame)       

In [8]:
from sportwarz import render_map, reset_county_styles, heatmap_counties

reset_county_styles(counties_geojson)
heatmap_counties(leagues, counties_geojson, "NFL")

render_map(leagues, "NFL", counties_geojson, co_data_frame)

HTML(value='\n        <style>\n        .leaflet-popup-content-wrapper {\n            background-color: black;\…

Map(center=[38.72728229549864, -96.9010842308538], controls=(ZoomControl(options=['position', 'zoom_in_text', …

In [9]:
from sportwarz import render_map, reset_county_styles, heatmap_counties

reset_county_styles(counties_geojson)
heatmap_counties(leagues, counties_geojson, "MLB") 

render_map(leagues, "MLB", counties_geojson, co_data_frame)

HTML(value='\n        <style>\n        .leaflet-popup-content-wrapper {\n            background-color: black;\…

Map(center=[38.72728229549864, -96.9010842308538], controls=(ZoomControl(options=['position', 'zoom_in_text', …

In [10]:
from sportwarz import render_map, reset_county_styles, heatmap_counties

reset_county_styles(counties_geojson)
heatmap_counties(leagues, counties_geojson, "NBA")

render_map(leagues, "NBA", counties_geojson, co_data_frame)

HTML(value='\n        <style>\n        .leaflet-popup-content-wrapper {\n            background-color: black;\…

Map(center=[38.72728229549864, -96.9010842308538], controls=(ZoomControl(options=['position', 'zoom_in_text', …

In [11]:
from sportwarz import render_map, reset_county_styles, heatmap_counties

reset_county_styles(counties_geojson)
heatmap_counties(leagues, counties_geojson, "NHL")

render_map(leagues, "NHL", counties_geojson, co_data_frame)

HTML(value='\n        <style>\n        .leaflet-popup-content-wrapper {\n            background-color: black;\…

Map(center=[38.72728229549864, -96.9010842308538], controls=(ZoomControl(options=['position', 'zoom_in_text', …

In [12]:
from sportwarz import render_map, reset_county_styles, heatmap_counties

reset_county_styles(counties_geojson)
heatmap_counties(leagues, counties_geojson, "MLS")

render_map(leagues, "MLS", counties_geojson, co_data_frame)

HTML(value='\n        <style>\n        .leaflet-popup-content-wrapper {\n            background-color: black;\…

Map(center=[38.72728229549864, -96.9010842308538], controls=(ZoomControl(options=['position', 'zoom_in_text', …